# Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import sys

### Importing Data as pandas dataframe and preprocessing it

In [2]:
df = pd.read_csv("AMZN 2010-2020.csv",error_bad_lines=False)

In [3]:
df = df.drop('High',axis=1)
df = df.drop('Low',axis=1)
df = df.drop('Adj Close',axis=1)
df = df.drop('Volume',axis=1)

In [4]:
df.describe()

,Open,Close
count,2517.000000,2517.000000
mean,755.984549,755.999992
std,633.737848,633.711304
min,105.930000,108.610001
25%,253.899994,253.369995
50%,430.070007,429.920013
75%,1141.000000,1152.349976
max,2500.000000,2497.939941


In [5]:
df.head()

,Date,Open,Close
0,2010-06-01,124.970001,123.239998
1,2010-06-02,124.019997,126.309998
2,2010-06-03,126.250000,128.759995
3,2010-06-04,126.330002,122.769997
4,2010-06-07,125.839996,122.010002


### Setting up the parameters for the model 

In [6]:
df['Difference'] = df['Close'].diff(-1)

In [7]:
df['Change Mod'] = df['Difference'].diff(-1).abs() / df['Open']

In [8]:
df.head()

,Date,Open,Close,Difference,Change Mod
0,2010-06-01,124.970001,123.239998,-3.070000,0.004961
1,2010-06-02,124.019997,126.309998,-2.449997,0.068054
2,2010-06-03,126.250000,128.759995,5.989998,0.041426
3,2010-06-04,126.330002,122.769997,0.759995,0.019077
4,2010-06-07,125.839996,122.010002,3.170006,0.017800


In [9]:
df.describe()

,Open,Close,Difference,Change Mod
count,2517.000000,2517.000000,2516.000000,2.515000e+03
mean,755.984549,755.999992,-0.921753,1.975634e-02
std,633.737848,633.711304,19.249531,2.011001e-02
min,105.930000,108.610001,-138.039917,7.060282e-08
25%,253.899994,253.369995,-5.349975,6.755031e-03
50%,430.070007,429.920013,-0.500000,1.416592e-02
75%,1141.000000,1152.349976,3.575003,2.626529e-02
max,2500.000000,2497.939941,187.959961,1.666971e-01


### Setting up the range  

In [10]:
criteria = [df['Difference'].between(-139, -df['Change Mod']*df['Open']/2), df['Difference'].between( -df['Change Mod']*df['Open']/2, df['Change Mod']*df['Open']/2), df['Difference'].between(df['Change Mod']*df['Open']/2,188)]
values = ['Down', 'Stagnant', 'Up']

df['Movement'] = np.select(criteria, values, 'Stagnant')

In [11]:
df.head()

,Date,Open,Close,Difference,Change Mod,Movement
0,2010-06-01,124.970001,123.239998,-3.070000,0.004961,Down
1,2010-06-02,124.019997,126.309998,-2.449997,0.068054,Stagnant
2,2010-06-03,126.250000,128.759995,5.989998,0.041426,Up
3,2010-06-04,126.330002,122.769997,0.759995,0.019077,Stagnant
4,2010-06-07,125.839996,122.010002,3.170006,0.017800,Up


### Showing number of States  

In [12]:
df['Movement'].value_counts()

Stagnant    913
Down        893
Up          711
Name: Movement, dtype: int64

In [13]:
z = list(df['Movement'])

### Transition Probability Matrix 

In [14]:
pd.crosstab(pd.Series(z[1:],name='Tomorrow'),
            pd.Series(z[:-1],name='Today'),normalize=0)

Today,Down,Stagnant,Up
Tomorrow,,,
Down,0.380045,0.455157,0.164798
Stagnant,0.434830,0.193866,0.371303
Up,0.220816,0.462729,0.316456


In [15]:
from itertools import islice

def window(seq, n=2):
    "Sliding window width n from seq.  From old itertools recipes."""
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [16]:
import pandas as pd

pairs = pd.DataFrame(window(z), columns=['Today', 'Tomorrow'])
counts = pairs.groupby('Today')['Tomorrow'].value_counts()
probs = (counts).unstack()

### Transition Count Matrix

In [17]:
probs

Tomorrow,Down,Stagnant,Up
Today,,,
Down,339,397,157
Stagnant,406,177,329
Up,147,339,225


# End